# NLP with CNN

### Exercise objectives:

- Use CNN instead of RNN for NLP

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-02-18 14:12:34.414720: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-18 14:12:34.446842: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Remember that to do NLP, you need to go through one of the following option, as shown here : 

<img src="embedding_or_RNN.png" width="700px" />

But in both cases, you can replace the recurrent layer (top part) by a CNN layer. We will go into both, starting by the one on the left were the embedding is learnt in the network.




# Part 1 : Concatenate a Keras Embedding with a Conv1D🔥 

Let's train some fancy network here. 

Each of your word is represented by a vector of size N (the size of your embedding). Therefore, as a sentence is a sequence of words, it is represented by a matrix (number of words, N). So, all your sentences are actually represented as matrices once embedded.

If you think about it, an image is also a matrix. Said differently, you may represent your sentence of word as a matrix, where each column (or row, depending on how you want to look at it) is a word, and each row (or each column) corresponds to a coordinate in the embedding space. As shown here

<img src="image_comparison.png" width="500px" />

Well, in that case, as these are close to images, why not using convolution on them? Yes, convolutions!
But, be careful. In the case of images, convolutions are 2 dimensional as the filters can move up and down, and left and right. In the case of our sentences, we want the corresponding kernel to move _only_ in the word by word direction (otherwise, moving coordinate of the embedding space by coordinate doesn't make much sense).

So let's create a model that use convolution

## First, the data

❓ **Question** ❓ You will need to prepare your data. First, tokenize them. Then, you need to pad them (use a value `maxlen` equal to 150). You also might need to compute the size of your vocabulary ;)

In [2]:
from tensorflow.keras import layers, models
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

In [8]:
vocab_size = len(tokenizer.word_index)

In [9]:
X_train_pad = pad_sequences(X_train_token, padding = 'post', dtype = 'float32', maxlen = 150)
X_test_pad = pad_sequences(X_test_token, padding = 'post', dtype = 'float32', maxlen = 150)


## Using 1D Convolution.

❓ **Question** ❓ Define a model that has :
- an `Embedding` layer: `input_dim` is the `vocab_size + 1`, `output_dim` is the embedding space dimension, and `mask_zero` has to be set to `True`. Here, for computational reasons, set `input_length` to the maximum length of your observations (that you just defined in the previous question).
- a `Conv1D` layer 
- a `Flatten` layer
- a `Dense` layer
- an output layer

Compile the model accordingly

❗ **Remark** ❗ The size of the `Conv1D` kernel corresponds exactly to the number of side-by-side words each kernel is taking into account ;)

In [19]:
model_cnn = models.Sequential()

model_cnn.add(layers.Embedding(input_dim = vocab_size+1,input_length=150, output_dim = 50, mask_zero = True))
model_cnn.add(layers.Conv1D(150 ,kernel_size = 2))

model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(50, activation = 'relu'))
model_cnn.add(layers.Dense(1, activation = 'sigmoid'))

model_cnn.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = "accuracy")

❓ **Question** ❓ Look at the number of parameters. You can compare it to the one that you had in previous exercise (esp. the first one)

In [20]:
model_cnn.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 150, 50)           1521000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 149, 150)          15150     
_________________________________________________________________
flatten_5 (Flatten)          (None, 22350)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                1117550   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 2,653,751
Trainable params: 2,653,751
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit your model with a stopping criterion, and evaluate it on the test data.

You will probably notice that it is ... **much faster** than RNN.

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_cnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=32,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_cnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
55/55 [==============================] - 1s 16ms/step - loss: 0.6894 - accuracy: 0.5303 - val_loss: 0.6682 - val_accuracy: 0.5653
Epoch 2/20
55/55 [==============================] - 1s 11ms/step - loss: 0.2653 - accuracy: 0.9274 - val_loss: 0.7072 - val_accuracy: 0.6480
Epoch 3/20
55/55 [==============================] - 1s 11ms/step - loss: 0.0236 - accuracy: 0.9960 - val_loss: 0.7343 - val_accuracy: 0.6507
Epoch 4/20
55/55 [==============================] - 1s 11ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.8097 - val_accuracy: 0.6707
Epoch 5/20
55/55 [==============================] - 1s 11ms/step - loss: 1.3929e-04 - accuracy: 1.0000 - val_loss: 0.9162 - val_accuracy: 0.6693
Epoch 6/20
55/55 [==============================] - 1s 12ms/step - loss: 8.5191e-06 - accuracy: 1.0000 - val_loss: 0.8962 - val_accuracy: 0.6907
The accuracy evaluated on the test set is of 53.160%


# Part 2 : Learn a Word2Vec representation, and then feed it to a NN with a `Conv1D`🔥 

In the first part of the exercise, you were asked to jointly learn the embedding representation and the CNN convolution within the same network, which was the CNN counterpart of the left part of this Figure : 

<img src="embedding_or_RNN.png" width="700px" />

Now, let's try to replace the RNN with a CNN for an architecture as on the right side.

❓ **Question** ❓ Learn a word2vec model or load it directly from the one trained in GENSIM (transfer learning). Then, prepare your data as in the previous exercise. This question is quite long, it prepares you to real world challenges - but you have already done all the building bricks in the previous exercises

In [22]:
import gensim.downloader as api
word2vec_transfer = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [23]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec_transfer, X_train)
X_test_embed = embedding(word2vec_transfer, X_test)

In [37]:
X_train_pad = pad_sequences(X_train_embed, dtype = 'float32', padding = 'post', maxlen = 200)
X_test_pad = pad_sequences(X_test_embed, dtype = 'float32', padding = 'post', maxlen = 200)

❓ **Question** ❓ Now construct a model that has a `Conv1D` layer, a flatten layer, a dense layer and an output layer, compile it and fit in on the train data. You can then evaluate it on the test set.

In [38]:
X_train_pad.shape

(2500, 200, 100)

In [41]:
model_rnn = models.Sequential()

model_rnn.add(layers.Masking(input_shape = (200,100)))
model_rnn.add(layers.Conv1D(200 ,kernel_size = 4))

model_rnn.add(layers.Flatten())
model_rnn.add(layers.Dense(50, activation = 'relu'))
model_rnn.add(layers.Dense(25, activation = 'relu'))
model_rnn.add(layers.Dense(1, activation = 'sigmoid'))

model_rnn.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = "accuracy")

In [42]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model_rnn.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=16,
          validation_split=0.3,
          callbacks=[es]
         )


res = model_rnn.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Epoch 1/20
110/110 [==============================] - 2s 15ms/step - loss: 0.9938 - accuracy: 0.5286 - val_loss: 3.4615 - val_accuracy: 0.5293
Epoch 2/20
110/110 [==============================] - 2s 15ms/step - loss: 0.6939 - accuracy: 0.7097 - val_loss: 0.7427 - val_accuracy: 0.6040
Epoch 3/20
110/110 [==============================] - 2s 14ms/step - loss: 0.3713 - accuracy: 0.8783 - val_loss: 3.6740 - val_accuracy: 0.4760
Epoch 4/20
110/110 [==============================] - 2s 16ms/step - loss: 0.1892 - accuracy: 0.9434 - val_loss: 1.1136 - val_accuracy: 0.5947
Epoch 5/20
110/110 [==============================] - 2s 16ms/step - loss: 0.0752 - accuracy: 0.9766 - val_loss: 1.4383 - val_accuracy: 0.5907
Epoch 6/20
110/110 [==============================] - 2s 15ms/step - loss: 0.0588 - accuracy: 0.9783 - val_loss: 1.6838 - val_accuracy: 0.5827
Epoch 7/20
110/110 [==============================] - 2s 15ms/step - loss: 0.0270 - accuracy: 0.9937 - val_loss: 2.3361 - val_accuracy: 0.6107

❓ **Question** ❓ You might been frustrated by the accuracy you got. This is where anyone gets at some point, when you have tested what came to your mind easily. Now, you need to improve your models, by making them more complex, changing the parameters, stacking additional layers, and so on.

Only practice and tests will get you through it. So you can come back to your previous models, change them and try to get better results ;)